In [1]:
import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy


from scipy.optimize import curve_fit

In [2]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]

energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [3]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [4]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [5]:
## Infinite cylinder parallel to z-axis

class ZCylinder:
    def __init__(self, x_0, y_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.r = r
        
    def distance(self, particle):
    
        a_ = particle.direction.tetta_x * particle.direction.tetta_x + (particle.direction.tetta_y * particle.direction.tetta_y )
        
        ## If a=0a 0a=0, this means the particle is parallel to the cylinder and will thus never intersect it
        if (abs(a_) < 1e-9):
            return -1
        
        c_ = ((particle.coordinates.x - self.x_0) * (particle.coordinates.x - self.x_0) + 
              (particle.coordinates.y - self.y_0) * (particle.coordinates.y - self.y_0) - self.r * self.r)

        k_ = (particle.coordinates.x -self.x_0) * particle.direction.tetta_x + (particle.coordinates.y -self.y_0) * particle.direction.tetta_y
        
        dec = k_ * k_ - a_ * c_
        
        if dec < 0:
            return -1
        
        dist_1 = (-k_ + math.sqrt(k_ * k_ - a_ * c_))/a_
        dist_2 = (-k_ - math.sqrt(k_ * k_ - a_ * c_))/a_
         
        distance = min(dist_1, dist_2)
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1
        
        
    def get_xml(self):

        obj_xml = " ZCylinder with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " r = "+ str(r) 
        
        return obj_xml

In [6]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [7]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [8]:
import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [9]:
def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [10]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [11]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [12]:
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = 0.
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False

In [13]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [14]:
def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        

In [15]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [16]:
def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, current_xs)
        set_random_direction(particle)
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)

        if is_collision_virual(particle, current_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions

In [17]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [18]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])

In [19]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [20]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [21]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [22]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [23]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [24]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     

In [25]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [26]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
        batch_particles = normalise_weights(batch_particles, initial_size) 

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [27]:
cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]

In [28]:
def compute_std(keff_cycle_exp):
    
    sample_std = []
    for i in range(2, len(keff_cycle_exp)):
        sample_std.append(np.std(keff_cycle_exp[:i]))
        
    return sample_std

In [29]:
test_number_of_particles = 20
test_number_interations = 500
test_number_inactive = 200

In [30]:
cylinder = ZCylinder(0, 0, 3.397610)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [cylinder, plane_5, plane_6]
signs = [-1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder_inside = Cell(surfaces, signs)

cylinder_inside.set_boundaries_type(boundaries_type)

cylinder_inside.set_box_sizes(-1000, -3.397610, -1000, 1000, 3.397610, 1000 )

cylinder_inside.set_zero_point(0., 0., 0.)

In [31]:
cylinder_in = ZCylinder(0, 0, 3.397610)
cylinder_out = ZCylinder(0, 0, 6.461335)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [ cylinder_out, cylinder_in, plane_5, plane_6]
signs = [-1, +1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder_outside = Cell(surfaces, signs)

cylinder_outside.set_boundaries_type(boundaries_type)

cylinder_outside.set_box_sizes(-1000, -6.461335, -1000, 1000, 6.461335, 1000 )

cylinder_outside.set_zero_point(0., 0., 0.)

In [32]:
cells = [cylinder_inside, cylinder_outside]
materials = [pu_23_cs_1_4, h2o_cs_1_4]

In [33]:
water_cylinder_universe = Universe(cells, materials)
estimators = []

In [34]:
test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 200

In [35]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(water_cylinder_universe,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, estimators)

 num1  7
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 0.9945188066631263   0.9883347243404219  0.0059165246701847
i == 258
 keff_cycle , k_effective_exp, std_k_effective 0.9877856830496924   0.9883252581112715  0.005813628445942412
i == 259
 keff_cycle , k_effective_exp, std_k_effective 0.9950691462568118   0.9884395613001788  0.005715385959425225
i == 260
 keff_cycle , k_effective_exp, std_k_effective 0.9831298429440237   0.9883510659942429  0.005620018997197536
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.0147696161437334   0.9887841569803001  0.005544061708869976
i == 262
 keff_cycle , k_effective_exp, std_k_effective 0.9510411720547468   0.9881753991589202  0.0054877775500110845
i == 263
 keff_cycle , k_effective_exp, std_k_effective 0.9418954744526655   0.9874407971794558  0.005449705355427479
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.0240824316065678   0.9880133227173794  0.00539434622434862
i == 265
 keff_cycle , k_effective_exp, std_k_effective 0.956080993

 keff_cycle , k_effective_exp, std_k_effective 1.0205064152803884   0.9947499515032149  0.004075950475993129
i == 329
 keff_cycle , k_effective_exp, std_k_effective 1.035526693467747   0.9950660502781338  0.0040565649329149635
i == 330
 keff_cycle , k_effective_exp, std_k_effective 1.0376181774796829   0.9953933743335304  0.0040385263628777805
i == 331
 keff_cycle , k_effective_exp, std_k_effective 0.9919893508377425   0.9953673894213488  0.004007663586108087
i == 332
 keff_cycle , k_effective_exp, std_k_effective 0.9978032628015494   0.9953858430075624  0.003977229425423121
i == 333
 keff_cycle , k_effective_exp, std_k_effective 1.0806088129447589   0.9960266172176163  0.003998884282800203
i == 334
 keff_cycle , k_effective_exp, std_k_effective 1.0860220887096899   0.9966982252138258  0.004025352163355954
i == 335
 keff_cycle , k_effective_exp, std_k_effective 0.9615446482721459   0.9964378283475911  0.00400390001213089
i == 336
 keff_cycle , k_effective_exp, std_k_effective 1.0126301

 keff_cycle , k_effective_exp, std_k_effective 1.073849004474514   0.9994853694979867  0.0032753214373924056
i == 399
 keff_cycle , k_effective_exp, std_k_effective 0.9650033523719606   0.9993120930300168  0.0032634244078079077
i == 400
 keff_cycle , k_effective_exp, std_k_effective 0.9733931711849366   0.9991824984207913  0.0032496514027756745
i == 401
 keff_cycle , k_effective_exp, std_k_effective 0.9876128058432972   0.9991249377612018  0.0032339558610912904
i == 402
 keff_cycle , k_effective_exp, std_k_effective 1.0364086524503633   0.9993095106061978  0.0032231953762705042
i == 403
 keff_cycle , k_effective_exp, std_k_effective 1.0170394429490073   0.9993968501743888  0.0032084672458878223
i == 404
 keff_cycle , k_effective_exp, std_k_effective 1.0187661277938487   0.9994917976136999  0.0031941122313165183
i == 405
 keff_cycle , k_effective_exp, std_k_effective 1.0429265654125597   0.9997036745297919  0.003185546989627269
i == 406
 keff_cycle , k_effective_exp, std_k_effective 1.0

 keff_cycle , k_effective_exp, std_k_effective 0.9675616880532872   1.0006593956873346  0.0027683989464954025
i == 469
 keff_cycle , k_effective_exp, std_k_effective 1.0307333507850458   1.0007711947769173  0.0027603532577236997
i == 470
 keff_cycle , k_effective_exp, std_k_effective 0.9383940579046147   1.000540168344057  0.0027597974940562534
i == 471
 keff_cycle , k_effective_exp, std_k_effective 1.0408640107068416   1.0006889648103403  0.0027536180555369455
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.0239395489010403   1.0007744448989089  0.0027448071351187644
i == 473
 keff_cycle , k_effective_exp, std_k_effective 0.9788436246893731   1.0006941122241486  0.002735914043969948
i == 474
 keff_cycle , k_effective_exp, std_k_effective 0.9765486317333846   1.0006059900325766  0.002727334686652721
i == 475
 keff_cycle , k_effective_exp, std_k_effective 1.0232219311074664   1.0006882298183035  0.0027186431821388216
i == 476
 keff_cycle , k_effective_exp, std_k_effective 1.00

In [36]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -75.2642150303906  with standart deviation [pcm]+- 264.1689005025144


In [53]:
cylinder = ZCylinder(0, 0, 3.077574)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [cylinder, plane_5, plane_6]
signs = [-1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder_inside = Cell(surfaces, signs)

cylinder_inside.set_boundaries_type(boundaries_type)

cylinder_inside.set_box_sizes(-1000, -3.077574, -1000, 1000, 3.077574, 1000 )

cylinder_inside.set_zero_point(0., 0., 0.)

In [54]:
cylinder_in = ZCylinder(0, 0, 3.077574)
cylinder_out = ZCylinder(0, 0, 33.714829)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [ cylinder_out, cylinder_in, plane_5, plane_6]
signs = [-1, +1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder_outside = Cell(surfaces, signs)

cylinder_outside.set_boundaries_type(boundaries_type)

cylinder_outside.set_box_sizes(-1000, -33.714829, -1000, 1000, 33.714829, 1000 )

cylinder_outside.set_zero_point(0., 0., 0.)

In [55]:
cells = [cylinder_inside, cylinder_outside]
materials = [pu_23_cs_1_4, h2o_cs_1_4]

In [59]:
water_cylinder_universe_2 = Universe(cells, materials)
estimators = []

In [60]:
test_number_of_particles = 20
test_number_interations = 500
test_number_inactive = 200

In [61]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(water_cylinder_universe_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_5, estimators)

 num1  4
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 1.0098583476635223   1.0015439328857123  0.0016492720794043026
i == 258
 keff_cycle , k_effective_exp, std_k_effective 0.9793467726136279   1.0011612232258489  0.0016651632290508162
i == 259
 keff_cycle , k_effective_exp, std_k_effective 1.0148139707640091   1.0013926257264958  0.0016529741502682997
i == 260
 keff_cycle , k_effective_exp, std_k_effective 0.9674362992495155   1.0008266869518794  0.0017209104528381487
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.0122659002763599   1.0010142150391659  0.0017028212280153958
i == 262
 keff_cycle , k_effective_exp, std_k_effective 0.996132052830983   1.000935470487421  0.0016769810227897958
i == 263
 keff_cycle , k_effective_exp, std_k_effective 1.011425770142957   1.001101983180366  0.0016585275284433
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.0037878249579422   1.0011439494581407  0.0016329467022052852
i == 265
 keff_cycle , k_effective_exp, std_k_effective 1.004044

 keff_cycle , k_effective_exp, std_k_effective 1.03612913087832   1.0001909555481552  0.0012218239531949352
i == 328
 keff_cycle , k_effective_exp, std_k_effective 0.9832210509553788   1.0000583781685242  0.0012194690213342238
i == 329
 keff_cycle , k_effective_exp, std_k_effective 0.9898190834965113   0.9999790037912217  0.0012125795227684694
i == 330
 keff_cycle , k_effective_exp, std_k_effective 1.0157793135205675   1.0001005446352937  0.001209338877422417
i == 331
 keff_cycle , k_effective_exp, std_k_effective 1.0023458829055765   1.0001176846220898  0.0012001941706957845
i == 332
 keff_cycle , k_effective_exp, std_k_effective 1.0194298585956223   1.0002639889703742  0.0012000190692472536
i == 333
 keff_cycle , k_effective_exp, std_k_effective 1.0250642152255816   1.0004504568369545  0.001205471363520048
i == 334
 keff_cycle , k_effective_exp, std_k_effective 0.9971449497831104   1.0004257888738664  0.0011966957605891728
i == 335
 keff_cycle , k_effective_exp, std_k_effective 0.981

 keff_cycle , k_effective_exp, std_k_effective 0.9924061833944525   1.000654416169553  0.0010166145027739623
i == 398
 keff_cycle , k_effective_exp, std_k_effective 0.9969135193756342   1.0006355227514017  0.0010116434962861487
i == 399
 keff_cycle , k_effective_exp, std_k_effective 1.0196075377030587   1.0007308595099527  0.001011051931012337
i == 400
 keff_cycle , k_effective_exp, std_k_effective 0.9947655882384999   1.0007010331535955  0.0010064260323865276
i == 401
 keff_cycle , k_effective_exp, std_k_effective 1.0008545195347875   1.0007017967674323  0.0010014067110266128
i == 402
 keff_cycle , k_effective_exp, std_k_effective 1.0070451270756424   1.0007331993927204  0.000996931622734874
i == 403
 keff_cycle , k_effective_exp, std_k_effective 1.001113171662555   1.0007350711773009  0.000992010239420583
i == 404
 keff_cycle , k_effective_exp, std_k_effective 0.9764051994440623   1.0006158071001772  0.000994314009639383
i == 405
 keff_cycle , k_effective_exp, std_k_effective 0.99150

 keff_cycle , k_effective_exp, std_k_effective 0.9915347737224289   1.0002361736567724  0.0008613477600078023
i == 468
 keff_cycle , k_effective_exp, std_k_effective 1.00892040084766   1.0002685774895743  0.000858739341071499
i == 469
 keff_cycle , k_effective_exp, std_k_effective 1.0204328746597677   1.0003435377021028  0.0008588186736613343
i == 470
 keff_cycle , k_effective_exp, std_k_effective 1.013757111050164   1.000393217603392  0.0008570730009388517
i == 471
 keff_cycle , k_effective_exp, std_k_effective 0.9954707347717504   1.0003750534601017  0.0008540976839660191
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.0089598753309816   1.0004066153052151  0.0008515369394865825
i == 473
 keff_cycle , k_effective_exp, std_k_effective 0.9995317462092103   1.0004034106565118  0.000848418075075126
i == 474
 keff_cycle , k_effective_exp, std_k_effective 0.9998279940359376   1.0004013105958527  0.0008453185962702094
i == 475
 keff_cycle , k_effective_exp, std_k_effective 0.98683

In [62]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -19.850602637272452  with standart deviation [pcm]+- 82.78665815235907


In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy


from scipy.optimize import curve_fit


# In[2]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]

energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  


# In[3]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[4]:


class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    


# In[5]:


## Infinite cylinder parallel to z-axis

class ZCylinder:
    def __init__(self, x_0, y_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.r = r
        
    def distance(self, particle):
    
        a_ = particle.direction.tetta_x * particle.direction.tetta_x + (particle.direction.tetta_y * particle.direction.tetta_y )
        
        ## If a=0a 0a=0, this means the particle is parallel to the cylinder and will thus never intersect it
        if (abs(a_) < 1e-9):
            return -1
        
        c_ = ((particle.coordinates.x - self.x_0) * (particle.coordinates.x - self.x_0) + 
              (particle.coordinates.y - self.y_0) * (particle.coordinates.y - self.y_0) - self.r * self.r)

        k_ = (particle.coordinates.x -self.x_0) * particle.direction.tetta_x + (particle.coordinates.y -self.y_0) * particle.direction.tetta_y
        
        dec = k_ * k_ - a_ * c_
        
        if dec < 0:
            return -1
        
        dist_1 = (-k_ + math.sqrt(k_ * k_ - a_ * c_))/a_
        dist_2 = (-k_ - math.sqrt(k_ * k_ - a_ * c_))/a_
         
        distance = min(dist_1, dist_2)
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1
        
        
    def get_xml(self):

        obj_xml = " ZCylinder with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " r = "+ str(r) 
        
        return obj_xml


# In[6]:


class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


# In[7]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[8]:


import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[9]:


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[10]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
         #   particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
          #  particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    


# In[11]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[12]:


def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = 0.
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False


# In[13]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)


# In[14]:


def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        


# In[15]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[16]:


def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, current_xs)
        set_random_direction(particle)
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)

        if is_collision_virual(particle, current_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions


# In[17]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[18]:


def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])


# In[19]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[20]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


# In[21]:


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[22]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[23]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  


# In[24]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     


# In[25]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[26]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
        batch_particles = normalise_weights(batch_particles, initial_size) 

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[27]:


cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]


# In[28]:


def compute_std(keff_cycle_exp):
    
    sample_std = []
    for i in range(2, len(keff_cycle_exp)):
        sample_std.append(np.std(keff_cycle_exp[:i]))
        
    return sample_std



estimators = []

cylinder = ZCylinder(0, 0, 3.077574)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [cylinder, plane_5, plane_6]
signs = [-1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder_inside = Cell(surfaces, signs)

cylinder_inside.set_boundaries_type(boundaries_type)

cylinder_inside.set_box_sizes(-1000, -3.077574, -1000, 1000, 3.077574, 1000 )

cylinder_inside.set_zero_point(0., 0., 0.)


cylinder_in = ZCylinder(0, 0, 3.077574)
cylinder_out = ZCylinder(0, 0, 33.714829)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [ cylinder_out, cylinder_in, plane_5, plane_6]
signs = [-1, +1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder_outside = Cell(surfaces, signs)

cylinder_outside.set_boundaries_type(boundaries_type)

cylinder_outside.set_box_sizes(-1000, -33.714829, -1000, 1000, 33.714829, 1000 )

cylinder_outside.set_zero_point(0., 0., 0.)


# In[55]:


cells = [cylinder_inside, cylinder_outside]
materials = [pu_23_cs_1_4, h2o_cs_1_4]


# In[59]:


water_cylinder_universe_2 = Universe(cells, materials)
estimators = []


# In[60]:


test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 200


# In[61]:


k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(water_cylinder_universe_2,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, estimators)


# In[62]:


difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[ ]:

def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)





 num1  0
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 1.0075940686396592   1.0009298486927365  0.005597319125744457
i == 259
 keff_cycle , k_effective_exp, std_k_effective 1.0243436293184214   1.0013266924321549  0.005515925405704719
i == 260
 keff_cycle , k_effective_exp, std_k_effective 1.0062382713825488   1.0014085520813283  0.005423831946212159
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.0795568758686909   1.0026896721434162  0.00548586351868933
i == 262
 keff_cycle , k_effective_exp, std_k_effective 1.0857714153689035   1.0040297002599563  0.005560537489445094
i == 263
 keff_cycle , k_effective_exp, std_k_effective 0.9670359515625587   1.00344249789968  0.005502981910289833
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.063242549062734   1.0043768736991028  0.005496319698670506
i == 265
 keff_cycle , k_effective_exp, std_k_effective 1.0480776926727937   1.0050491939910058  0.00545270765746901
i == 266
 keff_cycle , k_effective_exp, std_k_effective 0.997308947957

 keff_cycle , k_effective_exp, std_k_effective 1.0148269507886156   1.0030274328312987  0.0037766595365121202
i == 329
 keff_cycle , k_effective_exp, std_k_effective 0.9520771931082164   1.00263246973267  0.0037680258601823146
i == 330
 keff_cycle , k_effective_exp, std_k_effective 0.9554175678590557   1.0022692781797962  0.00375652710989062
i == 331
 keff_cycle , k_effective_exp, std_k_effective 1.0702999929006811   1.00278859661278  0.0037637408088584336
i == 332
 keff_cycle , k_effective_exp, std_k_effective 1.0174000566573107   1.0028992894919053  0.003736758661143387
i == 333
 keff_cycle , k_effective_exp, std_k_effective 0.9009973457117582   1.002133109463483  0.0037868749265936763
i == 334
 keff_cycle , k_effective_exp, std_k_effective 0.9919123680011361   1.0020568352734656  0.003759282287459446
i == 335
 keff_cycle , k_effective_exp, std_k_effective 0.9839974728168585   1.0019230622182314  0.0037337290437375347
i == 336
 keff_cycle , k_effective_exp, std_k_effective 0.96721547

 keff_cycle , k_effective_exp, std_k_effective 1.0210936085407798   1.0016520565397777  0.003047708693708522
i == 399
 keff_cycle , k_effective_exp, std_k_effective 1.1134607034071984   1.002213909036599  0.0030839673258415863
i == 400
 keff_cycle , k_effective_exp, std_k_effective 0.9711296843585321   1.0020584879132086  0.0030724422936461854
i == 401
 keff_cycle , k_effective_exp, std_k_effective 0.9727691321274619   1.00191276972522  0.0030605891701544754
i == 402
 keff_cycle , k_effective_exp, std_k_effective 1.0854263962015718   1.0023262035196572  0.0030733351519660976
i == 403
 keff_cycle , k_effective_exp, std_k_effective 1.0794617251784462   1.0027061814588631  0.003081673923587571
i == 404
 keff_cycle , k_effective_exp, std_k_effective 0.9229257255683192   1.002315100792733  0.0030913675319905105
i == 405
 keff_cycle , k_effective_exp, std_k_effective 0.9891894526709638   1.0022510732409193  0.003076916976406152
i == 406
 keff_cycle , k_effective_exp, std_k_effective 0.982268

 keff_cycle , k_effective_exp, std_k_effective 1.1018454263088326   1.0004697524367177  0.00270705145591281
i == 469
 keff_cycle , k_effective_exp, std_k_effective 1.002816462300502   1.0004784762652077  0.0026969834019104867
i == 470
 keff_cycle , k_effective_exp, std_k_effective 0.9937286555116076   1.0004534769290834  0.002687092300890407
i == 471
 keff_cycle , k_effective_exp, std_k_effective 1.0029832204007283   1.0004628117758423  0.002677174743651524
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.051323002244204   1.0006497977702111  0.002673860111351489
i == 473
 keff_cycle , k_effective_exp, std_k_effective 1.0241248888247654   1.0007357871147333  0.002665435155600513
i == 474
 keff_cycle , k_effective_exp, std_k_effective 1.052308699948155   1.0009240094243443  0.002662351257252469
i == 475
 keff_cycle , k_effective_exp, std_k_effective 0.9767444534057713   1.000836083766095  0.0026541091192327127
i == 476
 keff_cycle , k_effective_exp, std_k_effective 1.0324273990

In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy


from scipy.optimize import curve_fit


# In[2]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]

energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  


# In[3]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[4]:


class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    


# In[5]:


## Infinite cylinder parallel to z-axis

class ZCylinder:
    def __init__(self, x_0, y_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.r = r
        
    def distance(self, particle):
    
        a_ = particle.direction.tetta_x * particle.direction.tetta_x + (particle.direction.tetta_y * particle.direction.tetta_y )
        
        ## If a=0a 0a=0, this means the particle is parallel to the cylinder and will thus never intersect it
        if (abs(a_) < 1e-9):
            return -1
        
        c_ = ((particle.coordinates.x - self.x_0) * (particle.coordinates.x - self.x_0) + 
              (particle.coordinates.y - self.y_0) * (particle.coordinates.y - self.y_0) - self.r * self.r)

        k_ = (particle.coordinates.x -self.x_0) * particle.direction.tetta_x + (particle.coordinates.y -self.y_0) * particle.direction.tetta_y
        
        dec = k_ * k_ - a_ * c_
        
        if dec < 0:
            return -1
        
        dist_1 = (-k_ + math.sqrt(k_ * k_ - a_ * c_))/a_
        dist_2 = (-k_ - math.sqrt(k_ * k_ - a_ * c_))/a_
         
        distance = min(dist_1, dist_2)
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1
        
        
    def get_xml(self):

        obj_xml = " ZCylinder with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " r = "+ str(r) 
        
        return obj_xml


# In[6]:


class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


# In[7]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[8]:


import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[9]:


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[10]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
         #   particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
        #    particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    


# In[11]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[12]:


def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = 0.
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False


# In[13]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)


# In[14]:


def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        


# In[15]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[16]:


def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, current_xs)
        set_random_direction(particle)
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)

        if is_collision_virual(particle, current_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions


# In[17]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[18]:


def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])


# In[19]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[20]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


# In[21]:


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[22]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[23]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  


# In[24]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std     


# In[25]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[26]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
        batch_particles = normalise_weights(batch_particles, initial_size) 

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[27]:


cs_fission_pu_239 = [0.081600]

cs_fission_h2o = [0.0]

cs_capture_pu_239 = [0.019584]

cs_capture_h2o = [0.032640]

cs_scattering_pu_239 = [0.225216]

cs_scattering_h2o = [0.293760]

cs_total_pu_23 = [0.32640]

cs_total_h2o = [0.32640]

cs_production_neutrons_pu_239 = [2.84]

cs_production_neutrons_h2o = [0.0]

pu_23_cs_1_4 = Sig(1, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_total_pu_23)

h2o_cs_1_4 = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]


# In[28]:


def compute_std(keff_cycle_exp):
    
    sample_std = []
    for i in range(2, len(keff_cycle_exp)):
        sample_std.append(np.std(keff_cycle_exp[:i]))
        
    return sample_std



cylinder = ZCylinder(0, 0, 3.397610)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [cylinder, plane_5, plane_6]
signs = [-1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder_inside = Cell(surfaces, signs)

cylinder_inside.set_boundaries_type(boundaries_type)

cylinder_inside.set_box_sizes(-1000, -3.397610, -1000, 1000, 3.397610, 1000 )

cylinder_inside.set_zero_point(0., 0., 0.)


# In[31]:


cylinder_in = ZCylinder(0, 0, 3.397610)
cylinder_out = ZCylinder(0, 0, 6.461335)

plane_5 = Plane(0, 0, 1, 1000)
plane_6 = Plane(0, 0, 1, -1000)

surfaces = [ cylinder_out, cylinder_in, plane_5, plane_6]
signs = [-1, +1, -1, +1]
boundaries_type = ["black", "reflective", "reflective"]

cylinder_outside = Cell(surfaces, signs)

cylinder_outside.set_boundaries_type(boundaries_type)

cylinder_outside.set_box_sizes(-1000, -6.461335, -1000, 1000, 6.461335, 1000 )

cylinder_outside.set_zero_point(0., 0., 0.)


# In[32]:


cells = [cylinder_inside, cylinder_outside]
materials = [pu_23_cs_1_4, h2o_cs_1_4]


# In[33]:


water_cylinder_universe = Universe(cells, materials)
estimators = []


# In[35]:


test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 200


# In[37]:


k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(water_cylinder_universe,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, pu_23_cs_1_4, estimators)


# In[38]:


difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)



 num1  2
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 1.0139672497045096   0.9992323911126006  0.004791011302292141
i == 260
 keff_cycle , k_effective_exp, std_k_effective 0.9937066924010115   0.9991402961340742  0.0047113845601695955
i == 261
 keff_cycle , k_effective_exp, std_k_effective 0.9116948678784838   0.9977067645233267  0.004850194029538109
i == 262
 keff_cycle , k_effective_exp, std_k_effective 0.9220416059324293   0.9964863587396026  0.00492492856444142
i == 263
 keff_cycle , k_effective_exp, std_k_effective 0.958871803561759   0.9958893023082082  0.00488276563470686
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.0350078135399676   0.9965005290462045  0.004844579921015572
i == 265
 keff_cycle , k_effective_exp, std_k_effective 0.9778661888923478   0.9962138468899912  0.0047780737562563595
i == 266
 keff_cycle , k_effective_exp, std_k_effective 1.0460729402341504   0.9969692876982361  0.0047653815998249536
i == 267
 keff_cycle , k_effective_exp, std_k_effective 1.0035759

 keff_cycle , k_effective_exp, std_k_effective 1.0737981330761852   1.0003507299324743  0.0036158131701287226
i == 331
 keff_cycle , k_effective_exp, std_k_effective 0.9974080653514139   1.0003282668440694  0.003588175694826138
i == 332
 keff_cycle , k_effective_exp, std_k_effective 1.032247681377171   1.000570080590532  0.003569089922425568
i == 333
 keff_cycle , k_effective_exp, std_k_effective 0.9892922403978259   1.0004852847996093  0.0035431678297924924
i == 334
 keff_cycle , k_effective_exp, std_k_effective 0.9973163735294835   1.0004616362080414  0.0035167063881473047
i == 335
 keff_cycle , k_effective_exp, std_k_effective 0.9980013469777324   1.0004434118433723  0.0034906070843379843
i == 336
 keff_cycle , k_effective_exp, std_k_effective 1.014946411658476   1.0005500515478953  0.0034664864640327896
i == 337
 keff_cycle , k_effective_exp, std_k_effective 0.8819890773509964   0.9996846429771149  0.0035482441611819856
i == 338
 keff_cycle , k_effective_exp, std_k_effective 1.0331

 keff_cycle , k_effective_exp, std_k_effective 1.054041240240778   0.9996473527147235  0.002908255513399063
i == 402
 keff_cycle , k_effective_exp, std_k_effective 1.0284152455110567   0.9997897680255965  0.002897324657590169
i == 403
 keff_cycle , k_effective_exp, std_k_effective 1.0170707074447174   0.9998748958059863  0.0028842733181529885
i == 404
 keff_cycle , k_effective_exp, std_k_effective 0.958099245175216   0.9996701132048551  0.002877396278696218
i == 405
 keff_cycle , k_effective_exp, std_k_effective 0.9891088473711719   0.9996185948349348  0.0028637892313821546
i == 406
 keff_cycle , k_effective_exp, std_k_effective 1.0291640871096674   0.999762019554715  0.002853460223884101
i == 407
 keff_cycle , k_effective_exp, std_k_effective 0.9828141045265751   0.9996801455690718  0.0028408220041402014
i == 408
 keff_cycle , k_effective_exp, std_k_effective 1.0027260819873385   0.9996947894941596  0.002827169141920232
i == 409
 keff_cycle , k_effective_exp, std_k_effective 0.9921427

 keff_cycle , k_effective_exp, std_k_effective 1.0521475863396372   1.0008843731669035  0.0026144797511543348
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.0641773511205213   1.0011170679387917  0.0026152227753141744
i == 473
 keff_cycle , k_effective_exp, std_k_effective 1.0058605850046105   1.0011344434591793  0.0026056835285160787
i == 474
 keff_cycle , k_effective_exp, std_k_effective 1.0336836724512428   1.0012532362657196  0.0025988727085060775
i == 475
 keff_cycle , k_effective_exp, std_k_effective 0.9494827003434645   1.001064979771457  0.0025962393666915185
i == 476
 keff_cycle , k_effective_exp, std_k_effective 0.9539116582433719   1.0008941344036018  0.0025924511729582037
i == 477
 keff_cycle , k_effective_exp, std_k_effective 0.9989356335482565   1.0008870640034018  0.0025830848651745854
i == 478
 keff_cycle , k_effective_exp, std_k_effective 0.9407218778496711   1.0006706424704748  0.00258285952871308
i == 479
 keff_cycle , k_effective_exp, std_k_effective 1.05